In [1]:
import numpy as np
import sympy as sp

def snf(D):
    from sympy.matrices.normalforms import smith_normal_form
    return smith_normal_form(D,domain=sp.ZZ)

def ker(D):
    try:
        return sp.Matrix(np.hstack(D.nullspace()))
    except:
        return sp.Matrix([])

def im(D):
    try:
        return sp.Matrix(np.hstack(D.columnspace()))
    except:
        return sp.Matrix([])

def j_im_ker(D):
    assert abs(np.array(D*D)).max() == 0, "im sub ker not satisfied"
    Z,B = ker(D),im(D)
    # since im(D) sub ker(D), we can develop the image of B in terms
    # of Z, i.e. the matrix representing a Z-basis must be able to uniquely represent B
    # representing, because generating system for Z and B sub Z, uniquely, because basis
    T = Z.solve(B)
    return T

def pfin_achain_bdy(mflags):
    def flags_from_max_flags(mflags):
        def powerset(iterable):
            from itertools import chain, combinations
            "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
            s = list(iterable)
            return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))    
        fl = mflags.copy()
        all_flags = mflags.copy()
        for flag in mflags:
            contained_flags = powerset(flag)
            all_flags += contained_flags
            all_flags = sorted(set(all_flags),key=lambda x: (-len(x),x))
        return all_flags    
    def boundary_from_flags(flags):
        flags = sorted(set(flags),key=lambda x: (-len(x),x))
        A = np.zeros((len(flags),len(flags)),dtype=np.int8)
        flag_dict = dict(enumerate(flags))
        for i,flag in enumerate(flags):            
            for j in range(len(flag)):
                sign = (-1)**j; bdj = tuple(sorted(list(flag[:j])+list(flag[j+1:])))
                t_ix = flags.index(bdj)
                A[t_ix,i] = sign
        A = sp.Matrix(A)
        return A, flag_dict
    def snf_to_repr(dt):
        A = np.array(dt) - np.where( np.eye(*(dt.shape))>0, dt, 0)
        assert abs(A).max() == 0, "A was not diagonal, definitely not in snf"
        #now we know dt is diagonal (possibly/usually proper rectangular)
        ds = []
        adt = np.array(dt)
        for i in range(min(dt.shape)):
            if abs(int(adt[i,i])) > 1: #isomorphic base element B -> Z don't interest us, those are exactly the trivial cycles
                ds += [abs(int(adt[i,i]))]
        torsion_part = ds
        free_rank = max(dt.shape) - min(dt.shape) 
        return torsion_part,free_rank
    D, flag_dict = boundary_from_flags(flags_from_max_flags(mflags))
    T = j_im_ker(D)
    dT = snf_to_repr(snf(T))
    return dT
    
rp2_antichain = [(0,1,5),(1,2,5),(2,5,6),(0,2,6),(0,1,6),(1,4,6),(1,2,4),(0,2,4),(0,4,5),(4,5,6)]
dt = pfin_achain_bdy(rp2_antichain)
print(dt)

s1_antichain = [(0,2),(0,3),(1,2),(1,3)]
dt = pfin_achain_bdy(s1_antichain)
print(dt)
    
#s2 = dag.sphere(2)
#>>> s2
#max_flags: 0_2_4 0_2_5 0_3_4 0_3_5 1_2_4 1_2_5 1_3_4 1_3_5

s2_antichain = [(0,2,4),(0,2,5),(0,3,4),(0,3,5),(1,2,4),(1,2,5),(1,3,4),(1,3,5)]
dt = pfin_achain_bdy(s2_antichain)
print(dt)

s3_antichain = [(0,2,4,6),(0,2,4,7),(0,2,5,6),(0,2,5,7),(0,3,4,6),(0,3,4,7),(0,3,5,6),(0,3,5,7),(1,2,4,6),(1,2,4,7),(1,2,5,6),(1,2,5,7),(1,3,4,6),(1,3,4,7),(1,3,5,6),(1,3,5,7)]
dt = pfin_achain_bdy(s3_antichain)
print(dt)

#s4 = dag.sphere(4);  
#0_2_4_6_8 0_2_4_6_9 0_2_4_7_8 0_2_4_7_9 0_2_5_6_8 
#0_2_5_6_9 0_2_5_7_8 0_2_5_7_9 0_3_4_6_8 0_3_4_6_9 
#0_3_4_7_8 0_3_4_7_9 0_3_5_6_8 0_3_5_6_9 0_3_5_7_8 
#0_3_5_7_9 1_2_4_6_8 1_2_4_6_9 1_2_4_7_8 1_2_4_7_9 
#1_2_5_6_8 1_2_5_6_9 1_2_5_7_8 1_2_5_7_9 1_3_4_6_8 
#1_3_4_6_9 1_3_4_7_8 1_3_4_7_9 1_3_5_6_8 1_3_5_6_9 1_3_5_7_8 1_3_5_7_9
s4_antichain = [(0,2,4,6,8),(0,2,4,6,9),(0,2,4,7,8),(0,2,4,7,9),(0,2,5,6,8),(0,2,5,6,9),(0,2,5,7,8),(0,2,5,7,9),(0,3,4,6,8),(0,3,4,6,9),(0,3,4,7,8),(0,3,4,7,9),(0,3,5,6,8),(0,3,5,6,9),(0,3,5,7,8),(0,3,5,7,9),(1,2,4,6,8),(1,2,4,6,9),(1,2,4,7,8),(1,2,4,7,9),(1,2,5,6,8),(1,2,5,6,9),(1,2,5,7,8),(1,2,5,7,9),(1,3,4,6,8),(1,3,4,6,9),(1,3,4,7,8),(1,3,4,7,9),(1,3,5,6,8),(1,3,5,6,9),(1,3,5,7,8),(1,3,5,7,9),(0,2,4,6,8),(0,2,4,6,9),(0,2,4,7,8),(0,2,4,7,9),(0,2,5,6,8),(0,2,5,6,9),(0,2,5,7,8),(0,2,5,7,9),(0,3,4,6,8),(0,3,4,6,9),(0,3,4,7,8),(0,3,4,7,9),(0,3,5,6,8),(0,3,5,6,9),(0,3,5,7,8),(0,3,5,7,9),(1,2,4,6,8),(1,2,4,6,9),(1,2,4,7,8),(1,2,4,7,9),(1,2,5,6,8),(1,2,5,6,9),(1,2,5,7,8),(1,2,5,7,9),(1,3,4,6,8),(1,3,4,6,9),(1,3,4,7,8),(1,3,4,7,9),(1,3,5,6,8),(1,3,5,6,9),(1,3,5,7,8),(1,3,5,7,9)]
dt = pfin_achain_bdy(s4_antichain)
print(dt)

# SLOW
#t3_antichain = [(0,2,10,42),(0,2,10,58),(0,2,14,46),(0,2,14,62),(0,2,34,42),(0,2,34,46),(0,2,50,58),(0,2,50,62),(0,3,11,43),(0,3,11,59),(0,3,15,47),(0,3,15,63),(0,3,35,43),(0,3,35,47),(0,3,51,59),(0,3,51,63),(0,8,10,42),(0,8,10,58),(0,8,11,43),(0,8,11,59),(0,8,40,42),(0,8,40,43),(0,8,56,58),(0,8,56,59),(0,12,14,46),(0,12,14,62),(0,12,15,47),(0,12,15,63),(0,12,44,46),(0,12,44,47),(0,12,60,62),(0,12,60,63),(0,32,34,42),(0,32,34,46),(0,32,35,43),(0,32,35,47),(0,32,40,42),(0,32,40,43),(0,32,44,46),(0,32,44,47),(0,48,50,58),(0,48,50,62),(0,48,51,59),(0,48,51,63),(0,48,56,58),(0,48,56,59),(0,48,60,62),(0,48,60,63),(1,2,10,42),(1,2,10,58),(1,2,14,46),(1,2,14,62),(1,2,34,42),(1,2,34,46),(1,2,50,58),(1,2,50,62),(1,3,11,43),(1,3,11,59),(1,3,15,47),(1,3,15,63),(1,3,35,43),(1,3,35,47),(1,3,51,59),(1,3,51,63),(1,9,10,42),(1,9,10,58),(1,9,11,43),(1,9,11,59),(1,9,41,42),(1,9,41,43),(1,9,57,58),(1,9,57,59),(1,13,14,46),(1,13,14,62),(1,13,15,47),(1,13,15,63),(1,13,45,46),(1,13,45,47),(1,13,61,62),(1,13,61,63),(1,33,34,42),(1,33,34,46),(1,33,35,43),(1,33,35,47),(1,33,41,42),(1,33,41,43),(1,33,45,46),(1,33,45,47),(1,49,50,58),(1,49,50,62),(1,49,51,59),(1,49,51,63),(1,49,57,58),(1,49,57,59),(1,49,61,62),(1,49,61,63),(4,6,10,42),(4,6,10,58),(4,6,14,46),(4,6,14,62),(4,6,38,42),(4,6,38,46),(4,6,54,58),(4,6,54,62),(4,7,11,43),(4,7,11,59),(4,7,15,47),(4,7,15,63),(4,7,39,43),(4,7,39,47),(4,7,55,59),(4,7,55,63),(4,8,10,42),(4,8,10,58),(4,8,11,43),(4,8,11,59),(4,8,40,42),(4,8,40,43),(4,8,56,58),(4,8,56,59),(4,12,14,46),(4,12,14,62),(4,12,15,47),(4,12,15,63),(4,12,44,46),(4,12,44,47),(4,12,60,62),(4,12,60,63),(4,36,38,42),(4,36,38,46),(4,36,39,43),(4,36,39,47),(4,36,40,42),(4,36,40,43),(4,36,44,46),(4,36,44,47),(4,52,54,58),(4,52,54,62),(4,52,55,59),(4,52,55,63),(4,52,56,58),(4,52,56,59),(4,52,60,62),(4,52,60,63),(5,6,10,42),(5,6,10,58),(5,6,14,46),(5,6,14,62),(5,6,38,42),(5,6,38,46),(5,6,54,58),(5,6,54,62),(5,7,11,43),(5,7,11,59),(5,7,15,47),(5,7,15,63),(5,7,39,43),(5,7,39,47),(5,7,55,59),(5,7,55,63),(5,9,10,42),(5,9,10,58),(5,9,11,43),(5,9,11,59),(5,9,41,42),(5,9,41,43),(5,9,57,58),(5,9,57,59),(5,13,14,46),(5,13,14,62),(5,13,15,47),(5,13,15,63),(5,13,45,46),(5,13,45,47),(5,13,61,62),(5,13,61,63),(5,37,38,42),(5,37,38,46),(5,37,39,43),(5,37,39,47),(5,37,41,42),(5,37,41,43),(5,37,45,46),(5,37,45,47),(5,53,54,58),(5,53,54,62),(5,53,55,59),(5,53,55,63),(5,53,57,58),(5,53,57,59),(5,53,61,62),(5,53,61,63),(16,18,26,42),(16,18,26,58),(16,18,30,46),(16,18,30,62),(16,18,34,42),(16,18,34,46),(16,18,50,58),(16,18,50,62),(16,19,27,43),(16,19,27,59),(16,19,31,47),(16,19,31,63),(16,19,35,43),(16,19,35,47),(16,19,51,59),(16,19,51,63),(16,24,26,42),(16,24,26,58),(16,24,27,43),(16,24,27,59),(16,24,40,42),(16,24,40,43),(16,24,56,58),(16,24,56,59),(16,28,30,46),(16,28,30,62),(16,28,31,47),(16,28,31,63),(16,28,44,46),(16,28,44,47),(16,28,60,62),(16,28,60,63),(16,32,34,42),(16,32,34,46),(16,32,35,43),(16,32,35,47),(16,32,40,42),(16,32,40,43),(16,32,44,46),(16,32,44,47),(16,48,50,58),(16,48,50,62),(16,48,51,59),(16,48,51,63),(16,48,56,58),(16,48,56,59),(16,48,60,62),(16,48,60,63),(17,18,26,42),(17,18,26,58),(17,18,30,46),(17,18,30,62),(17,18,34,42),(17,18,34,46),(17,18,50,58),(17,18,50,62),(17,19,27,43),(17,19,27,59),(17,19,31,47),(17,19,31,63),(17,19,35,43),(17,19,35,47),(17,19,51,59),(17,19,51,63),(17,25,26,42),(17,25,26,58),(17,25,27,43),(17,25,27,59),(17,25,41,42),(17,25,41,43),(17,25,57,58),(17,25,57,59),(17,29,30,46),(17,29,30,62),(17,29,31,47),(17,29,31,63),(17,29,45,46),(17,29,45,47),(17,29,61,62),(17,29,61,63),(17,33,34,42),(17,33,34,46),(17,33,35,43),(17,33,35,47),(17,33,41,42),(17,33,41,43),(17,33,45,46),(17,33,45,47),(17,49,50,58),(17,49,50,62),(17,49,51,59),(17,49,51,63),(17,49,57,58),(17,49,57,59),(17,49,61,62),(17,49,61,63),(20,22,26,42),(20,22,26,58),(20,22,30,46),(20,22,30,62),(20,22,38,42),(20,22,38,46),(20,22,54,58),(20,22,54,62),(20,23,27,43),(20,23,27,59),(20,23,31,47),(20,23,31,63),(20,23,39,43),(20,23,39,47),(20,23,55,59),(20,23,55,63),(20,24,26,42),(20,24,26,58),(20,24,27,43),(20,24,27,59),(20,24,40,42),(20,24,40,43),(20,24,56,58),(20,24,56,59),(20,28,30,46),(20,28,30,62),(20,28,31,47),(20,28,31,63),(20,28,44,46),(20,28,44,47),(20,28,60,62),(20,28,60,63),(20,36,38,42),(20,36,38,46),(20,36,39,43),(20,36,39,47),(20,36,40,42),(20,36,40,43),(20,36,44,46),(20,36,44,47),(20,52,54,58),(20,52,54,62),(20,52,55,59),(20,52,55,63),(20,52,56,58),(20,52,56,59),(20,52,60,62),(20,52,60,63),(21,22,26,42),(21,22,26,58),(21,22,30,46),(21,22,30,62),(21,22,38,42),(21,22,38,46),(21,22,54,58),(21,22,54,62),(21,23,27,43),(21,23,27,59),(21,23,31,47),(21,23,31,63),(21,23,39,43),(21,23,39,47),(21,23,55,59),(21,23,55,63),(21,25,26,42),(21,25,26,58),(21,25,27,43),(21,25,27,59),(21,25,41,42),(21,25,41,43),(21,25,57,58),(21,25,57,59),(21,29,30,46),(21,29,30,62),(21,29,31,47),(21,29,31,63),(21,29,45,46),(21,29,45,47),(21,29,61,62),(21,29,61,63),(21,37,38,42),(21,37,38,46),(21,37,39,43),(21,37,39,47),(21,37,41,42),(21,37,41,43),(21,37,45,46),(21,37,45,47),(21,53,54,58),(21,53,54,62),(21,53,55,59),(21,53,55,63),(21,53,57,58),(21,53,57,59),(21,53,61,62),(21,53,61,63)]
#dt = pfin_achain_bdy(t3_antichain)
#print(dt)

t2_antichain = [(0,2,10),(0,2,14),(0,3,11),(0,3,15),(0,8,10),(0,8,11),(0,12,14),(0,12,15),(1,2,10),(1,2,14),(1,3,11),(1,3,15),(1,9,10),(1,9,11),(1,13,14),(1,13,15),(4,6,10),(4,6,14),(4,7,11),(4,7,15),(4,8,10),(4,8,11),(4,12,14),(4,12,15),(5,6,10),(5,6,14),(5,7,11),(5,7,15),(5,9,10),(5,9,11),(5,13,14),(5,13,15)]
dt = pfin_achain_bdy(t2_antichain); print(dt)

# SLOW too, double suspension of a torus
#X_antichain = [(0,2,10,16,18),(0,2,10,16,19),(0,2,10,17,18),(0,2,10,17,19),(0,2,14,16,18),(0,2,14,16,19),(0,2,14,17,18),(0,2,14,17,19),(0,3,11,16,18),(0,3,11,16,19),(0,3,11,17,18),(0,3,11,17,19),(0,3,15,16,18),(0,3,15,16,19),(0,3,15,17,18),(0,3,15,17,19),(0,8,10,16,18),(0,8,10,16,19),(0,8,10,17,18),(0,8,10,17,19),(0,8,11,16,18),(0,8,11,16,19),(0,8,11,17,18),(0,8,11,17,19),(0,12,14,16,18),(0,12,14,16,19),(0,12,14,17,18),(0,12,14,17,19),(0,12,15,16,18),(0,12,15,16,19),(0,12,15,17,18),(0,12,15,17,19),(1,2,10,16,18),(1,2,10,16,19),(1,2,10,17,18),(1,2,10,17,19),(1,2,14,16,18),(1,2,14,16,19),(1,2,14,17,18),(1,2,14,17,19),(1,3,11,16,18),(1,3,11,16,19),(1,3,11,17,18),(1,3,11,17,19),(1,3,15,16,18),(1,3,15,16,19),(1,3,15,17,18),(1,3,15,17,19),(1,9,10,16,18),(1,9,10,16,19),(1,9,10,17,18),(1,9,10,17,19),(1,9,11,16,18),(1,9,11,16,19),(1,9,11,17,18),(1,9,11,17,19),(1,13,14,16,18),(1,13,14,16,19),(1,13,14,17,18),(1,13,14,17,19),(1,13,15,16,18),(1,13,15,16,19),(1,13,15,17,18),(1,13,15,17,19),(4,6,10,16,18),(4,6,10,16,19),(4,6,10,17,18),(4,6,10,17,19),(4,6,14,16,18),(4,6,14,16,19),(4,6,14,17,18),(4,6,14,17,19),(4,7,11,16,18),(4,7,11,16,19),(4,7,11,17,18),(4,7,11,17,19),(4,7,15,16,18),(4,7,15,16,19),(4,7,15,17,18),(4,7,15,17,19),(4,8,10,16,18),(4,8,10,16,19),(4,8,10,17,18),(4,8,10,17,19),(4,8,11,16,18),(4,8,11,16,19),(4,8,11,17,18),(4,8,11,17,19),(4,12,14,16,18),(4,12,14,16,19),(4,12,14,17,18),(4,12,14,17,19),(4,12,15,16,18),(4,12,15,16,19),(4,12,15,17,18),(4,12,15,17,19),(5,6,10,16,18),(5,6,10,16,19),(5,6,10,17,18),(5,6,10,17,19),(5,6,14,16,18),(5,6,14,16,19),(5,6,14,17,18),(5,6,14,17,19),(5,7,11,16,18),(5,7,11,16,19),(5,7,11,17,18),(5,7,11,17,19),(5,7,15,16,18),(5,7,15,16,19),(5,7,15,17,18),(5,7,15,17,19),(5,9,10,16,18),(5,9,10,16,19),(5,9,10,17,18),(5,9,10,17,19),(5,9,11,16,18),(5,9,11,16,19),(5,9,11,17,18),(5,9,11,17,19),(5,13,14,16,18),(5,13,14,16,19),(5,13,14,17,18),(5,13,14,17,19),(5,13,15,16,18),(5,13,15,16,19),(5,13,15,17,18),(5,13,15,17,19)]
#dt = pfin_achain_bdy(X_antichain); print(dt)

([2], 0)
([], 1)
([], 1)
([], 1)
([], 1)
([], 3)
([], 3)


In [2]:
#jetzt gilt
assert pfin_achain_bdy(s2_antichain) == pfin_achain_bdy(s1_antichain)
#need grading back